In [3]:
import pandas as pd
import os

In [4]:
whole_author_name_paper_ids = pd.read_pickle('./pkl/whole_author_name_paper_ids.pkl')
train_author_paper_ids = pd.read_pickle('./pkl/train_author_paper_ids.pkl')

In [5]:
author_pub_ids = whole_author_name_paper_ids[['author_id','paper_ids']].merge(train_author_paper_ids, 'left', 'author_id')

In [6]:
author_pub_ids['paper_ids_x_len'] = author_pub_ids['paper_ids_x'].apply(len)
author_pub_ids['paper_ids_y_len'] = author_pub_ids['paper_ids_y'].apply(lambda x: 0 if type(x) == float else len(x))

In [7]:
author_pub_ids['paper_ids'] = author_pub_ids.apply(lambda row: list(set(row['paper_ids_x']) | (set() if type(row['paper_ids_y']) == float else set(row['paper_ids_y']))), axis=1)

In [8]:
author_pub_ids['paper_ids_len'] = author_pub_ids['paper_ids'].apply(len)

In [9]:
author_pub_ids.drop(columns=['paper_ids_x', 'paper_ids_y', 'paper_ids_x_len', 'paper_ids_y_len'], inplace=True)

In [10]:
del author_pub_ids['paper_ids_len']

In [11]:
author_pub_ids.head()

,author_id,paper_ids
0,004mBKh6,"[1vUnZLES, 2aTAK9bL, S6rklGvj, iQkpCzIk, 5IbEO..."
1,008b5hIc,[nPuf0kYs]
2,00BAgHFr,"[kn6wo3ot, TuEkjf5i, 0f2awX7U]"
3,00M93JDI,"[18XgtEKv, E184nnqB, Og5mWBX0, QZVTqWmf, LnvQy..."
4,00ShLPmg,"[XS959gS3, kIQzm6Id, i4q3CtMZ, zhcI3TWd, nPHrb..."


In [13]:
from tqdm import tqdm_notebook

pub_col = ['abstract', 'keywords', 'title', 'venue', 'authors', 'orgs']
for pc in pub_col:
    print(pc)
    pub_info_embed = pd.read_pickle('./embed_bert/pub_info_%s.pkl' % pc)
    pub_info_embed = pub_info_embed.set_index('paper_id')
    dat = []
    for author_id, paper_ids in tqdm_notebook(author_pub_ids.values):
        d = []
        for pid in paper_ids:
            d.append(pub_info_embed.loc[pid, pc])
        dat.append(d)
    author_pub_embed = author_pub_ids[['author_id', 'paper_ids']]
    author_pub_embed[pc] = dat
    author_pub_embed.to_pickle('./embed_bert/author_pub_%s.pkl' % pc)

abstract



keywords



title



venue



authors



orgs
